In [ ]:
#專家寫的code
# coding=utf-8
import requests
import json
import time
#import mysql.connector
import config
import datetime
import sys
from dateutil import parser

#cnx = mysql.connector.connect(host=config.db_host, user=config.db_user, passwd=config.db_password, database=config.db_database)
#cur = cnx.cursor()

def crawl(id_list,update_time):
	for post_id in id_list:
		while True:
			try:
				result = requests.get('https://www.dcard.tw/api/post/all/' + str(post_id)).json()
				parse_result(post_id,result,update_time)
				break
			except requests.exceptions.ConnectionError:
				print 'https://www.dcard.tw/api/post/all/' + str(post_id)
				print str(post_id) + ' error: ' + 'connection error.'
				time.sleep(5)
				continue

def parse_result(post_id,raw_data,update_time):
	#print raw_data
	if 'error' in raw_data and raw_data['error'] is True:
		print 'https://www.dcard.tw/api/post/all/' + str(post_id)
		print str(post_id) + ' error: ' + raw_data['msg']
		return
	write_post(post_id,raw_data,update_time)
	for comment in raw_data['comment']:
		write_comment(post_id,comment,update_time)


def write_post(post_id,post_data,update_time):
	print post_data.keys()
	sql = '''REPLACE INTO `dcard`.`posts` 
(`id`, 
`forum_alias`, 
`excerpt`, 
`pinned`, 
`anonymousDepartment`, 
`anonymousSchool`, 
`reply`, 
`updatedAt`, 
`createdAt`, 
`follow`, 
`commentCount`, 
`likeCount`, 
`like`, 
`title`, 
`content`, 
`author_gender`, 
`author_school`, 
`author_department`, 
`isLiked`, 
`currentUser`, 
`precious`,
`update_time`
)
VALUES
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
	id = post_id
	forum_alias = post_data['forum_alias']
	excerpt = post_data['excerpt']# if 'excerpt' in post_data else None
	pinned = post_data['pinned']# if 'pinned' in post_data else None
	anonymousDepartment = post_data['anonymousDepartment'] if 'anonymousDepartment' in post_data else None
	anonymousSchool = post_data['anonymousSchool'] if 'anonymousSchool' in post_data else None
	reply = post_data['reply']# if 'reply' in post_data else None
	updatedAt = parse_datetime(post_data['updatedAt'])
	createdAt = parse_datetime(post_data['createdAt'])# if 'createdAt' in post_data else None
	follow = post_data['follow']# if 'follow' in post_data else None
	commentCount = post_data['commentCount']# if 'commentCount' in post_data else None
	likeCount = post_data['likeCount']# if 'likeCount' in post_data else None
	like = post_data['like']# if 'like' in post_data else None
	title = post_data['version'][0]['title']# if 'title' in post_data['version'][0] else None
	content = post_data['version'][0]['content']# if 'content' in post_data['version'][0] else None
	author_gender = post_data['member']['gender']# if 'gender' in post_data['member'] else None
	author_school = post_data['member']['school']# if 'school' in post_data['member'] else None
	author_department = post_data['member']['department']# if 'department' in post_data['member'] else None
	isLiked = post_data['isLiked']# if 'isLiked' in post_data else None
	currentUser = post_data['currentUser']# if 'currentUser' in post_data else None
	precious = post_data['precious']# if 'precious' in post_data else None
	#cur.execute(sql,(id, forum_alias, excerpt, pinned, anonymousDepartment, anonymousSchool, reply, updatedAt, createdAt, follow, commentCount, likeCount, like, title, content, author_gender, author_school, author_department, isLiked, currentUser,precious,update_time))
	#cnx.commit()
    
	print content
	#for ele in post_data:
		#print ele,post_data[ele]
		#print "--------------------"
	print "*****************"

def write_comment(post_id,comment_data,update_time):
	sql = '''REPLACE INTO `dcard`.`comments` 
(`id`, 
`author_gender`, 
`author_school`, 
`author_department`, 
`content`, 
`createdAt`, 
`like`, 
`hidden`, 
`anonymous`, 
`host`, 
`num`, 
`currentUser`, 
`isLiked`, 
`post_id`, 
`update_time`
)
VALUES
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
	id = comment_data['_id']
	author_gender = comment_data['member']['gender'] if 'member' in comment_data else None
	author_school = comment_data['member']['school'] if 'member' in comment_data else None
	author_department = comment_data['member']['department'] if 'member' in comment_data else None
	content = comment_data['version'][0]['content']
	createdAt = parse_datetime(comment_data['version'][0]['createdAt'])
	like = comment_data['like']
	if not isinstance(like,int):
		like = len(like)
	hidden = comment_data['hidden']
	anonymous = comment_data['anonymous'] if 'anonymous' in comment_data else None
	host = comment_data['host'] if 'host' in comment_data else None
	num = comment_data['num'] if 'num' in comment_data else None
	currentUser = comment_data['currentUser'] if 'currentUser' in comment_data else None
	isLiked = comment_data['isLiked'] if 'isLiked' in comment_data else None
	#cur.execute(sql,(id,author_gender, author_school, author_department, content, createdAt, like, hidden, anonymous, host, num, currentUser, isLiked, post_id, update_time))
	#cnx.commit()
	for ele in comment_data:
		print ele, comment_data[ele]
		print "--------------------"
    
def parse_datetime(input_time):
	locale = datetime.timedelta(hours=8)
	if isinstance(input_time,int):
		return datetime.datetime.fromtimestamp(input_time/1000) + locale
	else:
		return parser.parse(input_time.replace(' ','')) + locale


if __name__ == '__main__':
	last_days = 7
	new_post_count = 100
	new_post_update_hours = 2
	now_datetime = datetime.datetime.now()
	page_list = []
	if sys.argv[1] == 'last':      
		if len(sys.argv) > 2:
			last_days = int(sys.argv[2])
		sql = 'SELECT id FROM posts WHERE createdAt>=%s AND createdAt<=%s'
		cur.execute(sql,(now_datetime - datetime.timedelta(days=last_days),now_datetime))
		results = cur.fetchall()
		for result in results:
			page_list.append(result[0])
	elif sys.argv[1] == 'new':  
		if len(sys.argv)> 2:
			new_post_count = sys.argv[2]
		#sql = 'SELECT MAX(id) FROM posts'
		#cur.execute(sql)
		max_id = cur.fetchone()[0]
		#sql = 'SELECT id FROM posts WHERE createdAt>=%s'
		#cur.execute(sql,(now_datetime - datetime.timedelta(hours=new_post_update_hours),))
		#results = cur.fetchall()
		for result in results:
			page_list.append(result[0])
		for new_id in range(max_id+1,max_id+new_post_count+1):
			page_list.append(new_id)
		#page_list = range(result[0]+1-new_post_count*2,result[0]+1+new_post_count)
	print len(page_list)
	update_time = datetime.datetime.now()
	print 'Start update('+sys.argv[1]+'): ' + str(update_time)
	page_list = range(30081, 30084)
	crawl(page_list,update_time)

## 試抓某一篇

In [8]:
import requests
import requests
import json
import time
#import mysql.connector
import datetime
import sys
from dateutil import parser

#https://www.dcard.tw/f/all/p/24318091

#文章藏在奇怪的地方，需要在後面使用.json()格式，把文章都變成字典，才有辦法得到內文
# url 也需要改
post_data = requests.get('https://www.dcard.tw/api/post/all/30083').json()
content = post_data['version'][0]['content']
member  = post_data["member"]#[0]#["department"]
for ele in post_data:
    print ele, post_data[ele]
print member['department'],member['gender'],member['school']

comment [{u'num': 0, u'report_reason': False, u'like': 1, u'isLiked': False, u'report_status': u'none', u'member': {u'department': u'\u6a5f\u68b0\u8207\u6a5f\u96fb\u5de5\u7a0b\u5b78\u7cfb', u'gender': u'M', u'school': u'\u6d77\u6d0b\u5927\u5b78'}, u'host': False, u'version': [{u'content': u'\u4f60\u90fd\u8aaa\u81ea\u5df1\u6709\u758f\u5931\u4e86\uff0c\n\u9084\u53eb\u5225\u4eba\u4e0d\u8981\u53bb?\n\n\u6d77\u5927 \u306e\u3073\u592a', u'createdAt': u'2014-08-23T15:12:54.417Z'}], u'anonymous': False, u'hidden': False, u'_id': u'53f8af760a4437481e1ae491', u'currentUser': False}]
anonymousSchool False
forum_name 閒聊
excerpt 今天小弟去中原吃飯 順便去影印店印筆記~~
重點來了~~有中
unreadComments 0
likeCount 0
commentCount 1
updatedAt 2014-08-23T15:12:54.417Z
follow False
id 30083
createdAt 2014-08-23T15:05:50.972Z
member {u'department': u'\u96fb\u6a5f\u5de5\u7a0b\u5b78\u7cfb', u'gender': u'M', u'school': u'\u5143\u667a\u5927\u5b78'}
isLiked False
pinned False
version [{u'content': u'\u4eca\u5929\u5c0f\u5f1f\u53bb\u4e2d\